# 梯度下降法

对于代价函数 $J(\theta)=\sum_{i=1}^{m}(\hat{y}-y)^2$,要使得其最小，应该沿着其 梯度 的 负方向 前进，因为梯度方向是下降最快的方向。    
梯度下降不是机器学习的模型，是一种常用的优化模型的算法。


## 批量梯度下降法（BGD）

一般而言，不做任何说明梯度下降法就是指批量梯度下降。    

批量梯度下降法：对所有变量求导，得到梯度方向，然后沿梯度方向进行迭代，直至达到局部最小。当函数为凸函数时，局部最小即为全局最小；但函数为非凸时，可能存在更小的局部最小。 

欲到达全局最小，可选取不同的初始点

 $\theta :=\theta -\eta \cdot \triangledown _{\theta }J(\theta )$  
其中 $\eta$ 为学习率，即梯度下降的步长；$\eta$ 过小收敛速度慢，过大可能无法达到局部最小，应该是一个合适的值。

$$\triangledown _{\theta }J(\theta_0 )=\frac{\sum_{i=0}^m(\hat{y}-y)^2}{m},\theta _0$$    
$$\triangledown _{\theta }J(\theta_i )=\frac{\sum_{i=0}^m(\hat{y}-y)^2\cdot x^i}{m},\theta _i$$
or $$\triangledown _{\theta }J(\theta )=\frac{X_b^T\cdot(X_b\cdot \theta-Y)}{m},X_b=(1,x_b^1,..,x_b^m)$$

In [1]:
import numpy as np


#代价函数
def J(x_b, y, theta):
    
    return np.sum((y - x_b.dot(theta))**2) / len(x_b)


#批量梯度下降
def dJ(theta, x_b, y):
    
#     result = np.empty(len(theta))
#     result[0] = np.sum(x_b.dot(theta) - y)
    
#     for i in range(1, len(theta)):
#         result[i] = (x_b.dot(theta) - y).dot(X_b[:, i])#矩阵点乘

#     return result * 2 / len(x_b)
    return X_b.T.dot(X_b.dot(theta) - y) * 2. / len(y)#向量化解法


def gd(i_theta, x_b, y, eta, n=1e4, e=1e-8):
    
    theta = i_theta
    j = 0
    
    while j < n:
#         求梯度
        gradient = dJ(theta, x_b, y)
    #         参数更新
        last_theta = theta
        theta =theta-eta * gradient
#         停止条件
        if (abs(J(x_b, y, theta) - J(x_b, y, last_theta)) < e):
            break 
        j += 1

    return theta


In [2]:
np.random.seed(666)
x = 2 * np.random.random(size=100000)
y = x * 3. + 4. + np.random.normal(size=100000)
X_b = np.hstack([np.ones((len(x), 1)), x.reshape(-1, 1)])
initial_theta = np.zeros(X_b.shape[1])
eta = 0.01

% time gd(initial_theta, X_b, y, eta)

Wall time: 1.92 s


array([3.99839758, 3.002347  ])

## 随机梯度下降法(SGD)

当数据量级很大时，采用批量梯度下降明显效率低下，为此可以采用随机梯度下降法。 

随机梯度下降：每次更新参数时，只沿一个方向更新；即每次只对一个变量求导，得到梯度方向。 

    优点：更新速度很快，有可能跳出局部最优到达全局最优。   
    缺点：不会精确收敛到全局最小值、参数方差波动大。
 
 更新参数：   
 $\theta :=\theta -\eta \cdot \triangledown _{\theta }J(\theta;x^{(i)},y^{(i)} )$
 
$$\triangledown _{\theta }J(\theta )=\frac{X_bi^T\cdot(X_bi\cdot \theta-Y)}{m},X_b=(1,x_b1,..,x_bm)$$

### 手写SGD

In [3]:
import numpy as np

# 代价函数
def dJ_sgd(theta, X_b_i, y_i):
    return 2 * X_b_i.T.dot(X_b_i.dot(theta) - y_i)


def sgd(X_b, y, initial_theta, n_iters):
#     初始化参数
    t0, t1 = 5, 50
#     定义学习率
    def learning_rate(t):
        return t0 / (t + t1)

    theta = initial_theta
    for cur_iter in range(n_iters):
#         随机索引
        rand_i = np.random.randint(len(X_b))
#         参数更新
        gradient = dJ_sgd(theta, X_b[rand_i], y[rand_i])
        theta = theta - learning_rate(cur_iter) * gradient

    return theta

In [4]:
np.random.seed(666)
x = 2 * np.random.random(size=100000)
y = x * 3. + 4. + np.random.normal(size=100000)
X_b = np.hstack([np.ones((len(x), 1)), x.reshape(-1, 1)])
initial_theta = np.zeros(X_b.shape[1])

% time sgd( X_b, y,initial_theta,n_iters=len(y)//3)

Wall time: 185 ms


array([3.99232723, 3.00189537])

### sklearn封装的SGD

[SGDClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier) 类实现了一个简单的随机梯度下降学习例程, 支持不同的 loss functions（损失函数）和 penalties for classification（分类处罚）。    



In [5]:
from sklearn.linear_model import SGDClassifier
X = [[0., 0.], [1., 1.]]
y = [0, 1]
clf = SGDClassifier(loss="hinge", penalty="l2")
clf.fit(X, y)

C:\Users\yep\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [6]:
clf.predict([[2., 2.]])

array([1])

In [7]:
clf.coef_

array([[9.91080278, 9.91080278]])

In [8]:
 clf.intercept_    

array([-9.99002993])

In [9]:
clf.decision_function([[2., 2.]])
#预测样本的置信度分数。

#样本的置信度得分是该样本与超平面的有符号距离。

array([29.65318117])

具体的 loss function（损失函数） 可以通过 loss 参数来设置。 SGDClassifier 支持以下的 loss functions（损失函数）：

- loss="hinge": (soft-margin) linear Support Vector Machine （（软-间隔）线性支持向量机），
- loss="modified_huber": smoothed hinge loss （平滑的 hinge 损失），
- loss="log": logistic regression （logistic 回归），
- and all regression losses below（以及所有的回归损失）。

使用 loss="log" 或者 loss="modified_huber" 来启用 predict_proba 方法, 其给出每个样本 x 的概率估计 P(y|x) 的一个向量：

In [10]:
>>> clf = SGDClassifier(loss="log").fit(X, y)

C:\Users\yep\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [11]:
>>> clf.predict_proba([[1., 1.]])

array([[4.97248476e-07, 9.99999503e-01]])

具体的惩罚方法可以通过 penalty 参数来设定。 SGD 支持以下 penalties（惩罚）:

- penalty="l2": L2 norm penalty on coef_.
- penalty="l1": L1 norm penalty on coef_.
- penalty="elasticnet": Convex combination of L2 and L1（L2 型和 L1 型的凸组合）; (1 - l1_ratio) * L2 + l1_ratio * L1.   
默认设置为 penalty="l2" 。 L1 penalty （惩罚）导致稀疏解，使得大多数系数为零。 Elastic Net（弹性网）解决了在特征高相关时 L1 penalty（惩罚）的一些不足。参数 l1_ratio 控制了 L1 和 L2 penalty（惩罚）的 convex combination （凸组合）。

**SGDClassifier 通过利用 “one versus all” （OVA）方法来组合多个二分类器，从而实现多分类。对于每一个 K 类, 可以训练一个二分类器来区分自身和其他 K-1 个类。在测试阶段，我们计算每个分类器的 confidence score（置信度分数）（也就是与超平面的距离），并选择置信度最高的分类。**

SGD允许使用minibatch（在线/核心）学习，请参阅[partial_fit](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier.partial_fit) 方法。



## 小批量梯度下降（MBGD）

小批量梯度下降法是BGD和SGD的折中，即保证了更新速度又减少了参数方差波动，一般mini-batch size在50到256之间，每次参数更新计算小批量样本的梯度。

 $\theta :=\theta -\eta \cdot \triangledown _{\theta }J(\theta;x^{(i:i+n)},y^{(i:i+n)} )$

 

In [12]:

def dJ_sgd(theta, X_b_i, y_i):
    return X_b_i.T.dot(X_b_i.dot(theta) - y_i)* 2. 

 
def mbgd(X_b, y, initial_theta, n_iters,n=100):
    
    t0 = 5
    t1 = 50
    
    def learning_rate(t):
        return t0/(t+t1)

    theta = initial_theta
    for cur_iter in range(1,n_iters): 
        for i in range(n):
#             随机索引
            rand_i= np.random.randint(len(X_b))
#             参数更新
            gradient = dJ_sgd(theta, X_b[rand_i], y[rand_i])
            theta = theta - learning_rate(cur_iter) * gradient
    return theta


In [13]:
np.random.seed(666)
x = 2 * np.random.random(size=100000)
y = x * 3. + 4. + np.random.normal(size=100000)
X_b = np.hstack([np.ones((len(x), 1)), x.reshape(-1, 1)])
initial_theta = np.zeros(X_b.shape[1])

% time mbgd( X_b, y,initial_theta,n_iters=len(y)//100)

Wall time: 550 ms


array([4.02667622, 2.94687564])

上述算法在数据量较大时候准确度较为稳定，可能是我没有设置退出条件,不过当数据量级达到一定程度时，准确度越来越高

# 梯度调试


$$\triangledown _{\theta }J(\theta_i )=\frac{J(\theta_i+\epsilon)-J(\theta_i-\epsilon)}{2\epsilon}$$

这种方法计算时间复杂度高，但能保证结论一定正确